<a href="https://colab.research.google.com/github/fragenabhishek/ImageClassification/blob/main/air_grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pandas as pd
import numpy as np
import sklearn


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "theano"
import keras.backend


In [ ]:
train_dataset = pd.read_csv("Train_Combine.csv")



In [ ]:
train_dataset.describe()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
count,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000
mean,0.147966,0.229383,0.142745,0.115035,0.108447,-0.631960,-0.492956,-0.627964,0.487013
std,0.465568,0.396136,0.464150,0.491129,0.411527,0.157452,0.337699,0.223215,0.500102
min,-1.000000,-0.844011,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,-0.266458,-0.052925,-0.296636,-0.326299,-0.128205,-0.724138,-0.741667,-0.750339,0.000000
50%,0.266458,0.320334,0.207951,0.172078,0.153846,-0.609195,-0.541667,-0.649932,0.000000
75%,0.529781,0.509749,0.565749,0.551948,0.384615,-0.517241,-0.291667,-0.554953,1.000000
max,1.000000,1.000000,0.932722,0.987013,0.948718,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Convert dataframe to numpy array
dataset = train_dataset.values
print(dataset.shape)

(924, 9)


In [ ]:
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [ ]:
# Transform and display the training data
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,9.240000e+02,9.240000e+02,9.240000e+02,9.240000e+02,9.240000e+02,9.240000e+02,9.240000e+02,9.240000e+02
mean,1.754249e-17,1.417817e-17,-5.671269e-17,2.499203e-17,5.935608e-17,-3.747603e-16,6.007700e-17,8.170472e-17
std,1.000542e+00,1.000542e+00,1.000542e+00,1.000542e+00,1.000542e+00,1.000542e+00,1.000542e+00,1.000542e+00
min,-2.467070e+00,-2.711131e+00,-2.463348e+00,-2.271580e+00,-2.694959e+00,-2.338739e+00,-1.502280e+00,-1.667616e+00
25%,-8.906299e-01,-7.130405e-01,-9.471474e-01,-8.990970e-01,-5.753709e-01,-5.857519e-01,-7.368845e-01,-5.485350e-01
50%,2.546469e-01,2.297208e-01,1.405618e-01,1.162097e-01,1.103784e-01,1.446594e-01,-1.443200e-01,-9.846986e-02
75%,8.205484e-01,7.081369e-01,9.118464e-01,8.900917e-01,6.714459e-01,7.289884e-01,5.963856e-01,3.272675e-01
max,1.831087e+00,1.946390e+00,1.702908e+00,1.776418e+00,2.042944e+00,1.037042e+01,4.423364e+00,7.297195e+00


In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [ ]:
# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Do a grid search for the optimal batch size and number of epochs
# import necessary packages
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, verbose = 1)

# define the grid search parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=10 ........................................


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Epoch 1/10
616/616 [==============================] - 0s 60us/step - loss: 0.4916 - acc: 0.8312
Epoch 2/10
616/616 [==============================] - 0s 45us/step - loss: 0.3579 - acc: 0.8506
Epoch 3/10
616/616 [==============================] - 0s 45us/step - loss: 0.3287 - acc: 0.8669
Epoch 4/10
616/616 [==============================] - 0s 45us/step - loss: 0.3269 - acc: 0.8653
Epoch 5/10
616/616 [==============================] - 0s 45us/step - loss: 0.3188 - acc: 0.8782
Epoch 6/10
616/616 [==============================] - 0s 48us/step - loss: 0.3147 - acc: 0.8799
Epoch 7/10
616/616 [==============================] - 0s 48us/step - loss: 0.3112 - acc: 0.8782
Epoch 8/10
616/616 [==============================] - 0s 47us/step - loss: 0.3114 - acc: 0.8718
Epoch 9/10
616/616 [==============================] - 0s 47us/step - loss: 0.3151 - acc: 0.8750
Epoch 10/10
308/308 [==============================] - 0s 19us/step
[CV] ............ batch_size=10, epochs=10, score=0.857, total=  26.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   26.4s remaining:    0.0s


...

Epoch 1/100
616/616 [==============================] - 0s 53us/step - loss: 0.5186 - acc: 0.8263
Epoch 2/100
616/616 [==============================] - 0s 49us/step - loss: 0.3426 - acc: 0.8620
Epoch 3/100
616/616 [==============================] - 0s 48us/step - loss: 0.3007 - acc: 0.8750
Epoch 4/100
616/616 [==============================] - 0s 47us/step - loss: 0.2946 - acc: 0.8766
Epoch 5/100
616/616 [==============================] - 0s 49us/step - loss: 0.2913 - acc: 0.8718
Epoch 6/100
616/616 [==============================] - 0s 49us/step - loss: 0.2923 - acc: 0.8750
Epoch 7/100
616/616 [==============================] - 0s 47us/step - loss: 0.2859 - acc: 0.8750
Epoch 8/100
616/616 [==============================] - 0s 47us/step - loss: 0.2901 - acc: 0.8685
Epoch 9/100
616/616 [==============================] - 0s 47us/step - loss: 0.2795 - acc: 0.8880
Epoch 10/100
616/616 [==============================] - 0s 47us/step - loss: 0.2838 - acc: 0.8701
Epoch 11/100
616/616 [=======

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   50.5s remaining:    0.0s


Epoch 1/10
616/616 [==============================] - 0s 30us/step - loss: 0.5498 - acc: 0.7597
Epoch 2/10
616/616 [==============================] - 0s 30us/step - loss: 0.3923 - acc: 0.8571
Epoch 3/10
616/616 [==============================] - 0s 28us/step - loss: 0.3534 - acc: 0.8571
Epoch 4/10
616/616 [==============================] - 0s 28us/step - loss: 0.3380 - acc: 0.8636
Epoch 5/10
616/616 [==============================] - 0s 27us/step - loss: 0.3234 - acc: 0.8701
Epoch 6/10
616/616 [==============================] - 0s 32us/step - loss: 0.3123 - acc: 0.8750
Epoch 7/10
616/616 [==============================] - 0s 36us/step - loss: 0.3133 - acc: 0.8750
Epoch 8/10
616/616 [==============================] - 0s 27us/step - loss: 0.3077 - acc: 0.8782
Epoch 9/10
616/616 [==============================] - 0s 35us/step - loss: 0.3034 - acc: 0.8766
Epoch 10/10
308/308 [==============================] - 0s 13us/step
[CV] ............ batch_size=20, epochs=10, score=0.847, total=   1.

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.5min finished


Epoch 1/100
924/924 [==============================] - 0s 14us/step - loss: 0.5863 - acc: 0.7857
Epoch 2/100
924/924 [==============================] - 0s 15us/step - loss: 0.3986 - acc: 0.8485
Epoch 3/100
924/924 [==============================] - 0s 14us/step - loss: 0.3545 - acc: 0.8582
Epoch 4/100
924/924 [==============================] - 0s 15us/step - loss: 0.3310 - acc: 0.8582
Epoch 5/100
924/924 [==============================] - 0s 16us/step - loss: 0.3270 - acc: 0.8690
Epoch 6/100
924/924 [==============================] - 0s 15us/step - loss: 0.3229 - acc: 0.8604
Epoch 7/100
924/924 [==============================] - 0s 15us/step - loss: 0.3181 - acc: 0.8626
Epoch 8/100
924/924 [==============================] - 0s 14us/step - loss: 0.3238 - acc: 0.8658
Epoch 9/100
924/924 [==============================] - 0s 14us/step - loss: 0.3145 - acc: 0.8680
Epoch 10/100
924/924 [==============================] - 0s 15us/step - loss: 0.3114 - acc: 0.8636
Epoch 11/100
924/924 [=======

In [ ]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.844, total=   2.2s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.903, total=   2.4s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.6s remaining:    0.0s


...

[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.825, total=   2.5s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   20.9s remaining:    0.0s


[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.851, total=   8.7s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.890, total=   3.1s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.834, total=   2.8s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.844, total=   3.1s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.886, total=   3.0s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.838, total=   2.8s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.1, learn_rate=0.1, score=0.847, total=   3.1s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.3min finished


Best: 0.864718615492701, using {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.8614718604397464 (0.029200724210761534) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.860389607551294 (0.018366404792240946) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.8387445913248764 (0.013603686989274603) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.8582251115794822 (0.023161180813890434) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.8560606047704622 (0.021590840478688034) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.8528138561682268 (0.015074014194739882) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.864718615492701 (0.022546174288237363) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.8614718630200341 (0.020072767723627122) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.8398268439553 (0.0170433062673635) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [ ]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.844, total=   2.1s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.903, total=   2.5s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.6s remaining:    0.0s


...

[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.825, total=   2.5s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   20.8s remaining:    0.0s


[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.851, total=   2.8s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.890, total=   3.1s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.834, total=   2.8s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.844, total=   3.1s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.886, total=   2.7s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.838, total=   3.1s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.1, learn_rate=0.1, score=0.847, total=   3.1s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.2min finished


Best: 0.864718615492701, using {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.8614718604397464 (0.029200724210761534) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.860389607551294 (0.018366404792240946) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.8387445913248764 (0.013603686989274603) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.8582251115794822 (0.023161180813890434) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.8560606047704622 (0.021590840478688034) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.8528138561682268 (0.015074014194739882) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.864718615492701 (0.022546174288237363) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.8614718630200341 (0.020072767723627122) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.8398268439553 (0.0170433062673635) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [ ]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(X_standardized)

In [ ]:
print(y_pred.shape)

(924, 1)


In [ ]:
print(y_pred[:5])

[[0]
 [1]
 [0]
 [0]
 [1]]


In [ ]:
# Generate a classification report
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.8647186147186147
              precision    recall  f1-score   support

           0       0.86      0.88      0.87       474
           1       0.87      0.84      0.86       450

    accuracy                           0.86       924
   macro avg       0.87      0.86      0.86       924
weighted avg       0.86      0.86      0.86       924



In [ ]:
test_dataset=pd.read_csv('Test_Combine.csv')

In [ ]:
dataset = test_dataset.values
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [ ]:
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,2.310000e+02,2.310000e+02,2.310000e+02,2.310000e+02,2.310000e+02,2.310000e+02,2.310000e+02,2.310000e+02
mean,-7.161179e-17,-7.954195e-17,-3.460435e-17,-6.536378e-17,-9.131705e-18,7.522842e-16,8.242565e-17,5.008019e-16
std,1.002172e+00,1.002172e+00,1.002172e+00,1.002172e+00,1.002172e+00,1.002172e+00,1.002172e+00,1.002172e+00
min,-2.514426e+00,-3.232382e+00,-2.012021e+00,-1.873667e+00,-2.725267e+00,-2.455342e+00,-1.607217e+00,-1.529572e+00
25%,-9.385959e-01,-7.026296e-01,-8.967126e-01,-9.566817e-01,-6.374679e-01,-6.854060e-01,-7.358221e-01,-5.487778e-01
50%,2.094069e-01,2.579430e-01,7.918205e-02,1.424000e-01,1.454568e-01,1.191102e-01,-1.020800e-01,-7.128581e-02
75%,8.083648e-01,7.272304e-01,8.947511e-01,8.382684e-01,7.000284e-01,7.627231e-01,7.165034e-01,3.803958e-01
max,1.763845e+00,1.929779e+00,1.919440e+00,1.807281e+00,2.168012e+00,2.210852e+00,3.225066e+00,5.168221e+00


In [ ]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.870, total=   1.3s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.896, total=   1.6s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.0s remaining:    0.0s


...

[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.844, total=   1.3s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   13.2s remaining:    0.0s


[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.857, total=   1.9s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.896, total=   1.6s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.857, total=   1.9s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.831, total=   2.0s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.844, total=   1.6s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.896, total=   1.9s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.1, learn_rate=0.1, score=0.844, total=   1.9s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   45.7s finished


Best: 0.8874458864137724, using {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.8831168823427968 (0.010603852648418677) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.8658008673490384 (0.022073675726293086) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.8484848471947046 (0.026685766826073574) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.8701298662594387 (0.018366414279842938) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.8571428566267996 (0.028055149880242564) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.8441558534448798 (8.361087611292674e-09) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.8874458864137724 (0.01619765212837301) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.8701298752904454 (0.0212077005565455) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.8701298662594387 (0.018366406069417696) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}
